## All the imports for necessary for reading the tiny_imagnet

In [12]:
import numpy as np
import os
import imageio
import glob
import re

In [13]:
# code from https://github.com/pat-coady/tiny_imagenet/blob/master/src/input_pipe.py
def load_filenames_labels(mode):
  """Gets filenames and labels

  Args:
    mode: 'train' or 'val'
      (Directory structure and file naming different for
      train and val datasets)

  Returns:
    list of tuples: (jpeg filename with path, label)
  """
  label_dict, class_description = build_label_dicts()
  filenames_labels = []
  if mode == 'train':
    filenames = glob.glob('tiny-imagenet-200/train/*/images/*.JPEG')
    for filename in filenames:
      match = re.search(r'n\d+', filename)
      label = str(label_dict[match.group()])
      filenames_labels.append((filename, label))
  elif mode == 'val':
    with open('tiny-imagenet-200/val/val_annotations.txt', 'r') as f:
      for line in f.readlines():
        split_line = line.split('\t')
        filename = 'tiny-imagenet-200/val/images/' + split_line[0]
        label = str(label_dict[split_line[1]])
        filenames_labels.append((filename, label))

  return filenames_labels


def build_label_dicts():
  """Build look-up dictionaries for class label, and class description

  Class labels are 0 to 199 in the same order as 
    tiny-imagenet-200/wnids.txt. Class text descriptions are from 
    tiny-imagenet-200/words.txt

  Returns:
    tuple of dicts
      label_dict: 
        keys = synset (e.g. "n01944390")
        values = class integer {0 .. 199}
      class_desc:
        keys = class integer {0 .. 199}
        values = text description from words.txt
  """
  label_dict, class_description = {}, {}
  with open('tiny-imagenet-200/wnids.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
      synset = line[:-1]  # remove \n
      label_dict[synset] = i
  with open('tiny-imagenet-200/words.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
      synset, desc = line.split('\t')
      desc = desc[:-1]  # remove \n
      if synset in label_dict:
        class_description[label_dict[synset]] = desc

  return label_dict, class_description

In [21]:
mode='train'
filenames_labels = load_filenames_labels(mode)
count_dict=[]
status_dict=[]
for i in range(200):
    count_dict.append(0)
    status_dict.append(0)
for f in filenames_labels:
    count_dict[int(f[1])] = count_dict[int(f[1])]+1
for i in range(200):
    image_tensor=np.ndarray(shape=(len(count_dict),count_dict[i],64,64,3), dtype='float64')
for f in filenames_labels:
    label = int(f[1])
    img = imageio.imread(f[0])    
    if(img.ndim==2):
        for i in range(3):
            image_tensor[label,status_dict[label],:,:,i] = img
    else:
        image_tensor[label,status_dict[label],:,:,:] = img
    status_dict[label]=status_dict[label]+1

In [22]:
image_tensor=image_tensor.reshape((image_tensor.shape[0],image_tensor.shape[1],image_tensor.shape[2],image_tensor.shape[3]*image_tensor.shape[4]))
print(image_tensor.shape)

(200, 500, 64, 192)


In [23]:
np.save('/Users/rnu/Documents/research/nmflibrary/test_data/tiny_imagenet_tensor.npy', image_tensor)